<div align="center" style="line-height: 1.2;">

### **BCSE417P Lab Assignment 3**

#### **Name**: SreeDananjay S

#### **Reg No**: 21BAI1807

##### **Nov 9, 2024**

##### By turning in this assignment, I agree and declare that all of this is my own work.

</div>

In [2]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 2.3 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.1.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.3 which is incompatible.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.1.3 which is incompatible.


#### Step 1: Video Loading
The first step is to load the provided video file using OpenCV’s `cv2.VideoCapture()` function, which reads and captures the video for processing. We also retrieve key video properties, such as frame count and frame rate, to understand the video’s structure and adjust processing based on its frame rate if needed.

In [10]:
import cv2

# Load video
video_path = './5738706-hd_1920_1080_24fps.mp4'
video = cv2.VideoCapture(video_path)

# Get video properties
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = video.get(cv2.CAP_PROP_FPS)
frame_count, frame_rate

(178, 23.976023976023978)

#### 2. **Frame Extraction**

#### Step 2: Frame Extraction
To analyze each frame individually, we extract frames from the video in a loop until all frames are read. Each frame is stored in a list for later processing. This is achieved by reading each frame in a `while` loop until there are no more frames to read (`success` becomes `False`).

In [11]:
frames = []
success, frame = video.read()
while success:
    frames.append(frame)
    success, frame = video.read()
video.release()

#### 3. **Spatio-Temporal Segmentation**

#### Step 3: Spatio-Temporal Segmentation
This step segments objects within each frame to observe changes over time. We use edge detection here (using the Canny edge detection method) to find object boundaries, but color thresholding or other techniques can also be applied. 
Each processed frame is added to `segmented_frames` for visualization and tracking.

In [12]:
segmented_frames = []
for frame in frames:
    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edges = cv2.Canny(gray_frame, threshold1=50, threshold2=150)
    segmented_frames.append(edges)

#### 4. **Scene Cut Detection**

#### Step 4: Scene Cut Detection
We use a histogram-based approach to detect abrupt and gradual scene transitions. By calculating and comparing histograms between consecutive frames, we identify frames where the difference is significant enough to mark a scene cut. A threshold is set to detect large changes (hard cuts) and gradually changing intensities (soft cuts).


In [13]:
scene_cuts = []
hist_prev = cv2.calcHist([frames[0]], [0], None, [256], [0, 256])

for i in range(1, len(frames)):
    hist_curr = cv2.calcHist([frames[i]], [0], None, [256], [0, 256])
    hist_diff = cv2.compareHist(hist_prev, hist_curr, cv2.HISTCMP_CORREL)

    if hist_diff < 0.5:  # Threshold for hard cut
        scene_cuts.append(i)

    # Update previous histogram
    hist_prev = hist_curr

#### 5. **Mark Scene Cuts**

#### Step 5: Mark Scene Cuts
Once the scene cuts are identified, we annotate frames where the cuts were detected to make them visually recognizable. We display a message like "Scene Cut" on these frames and store them in `scene_summary` for visual presentation.

In [14]:
scene_summary = []
for cut_frame in scene_cuts:
    summary_frame = frames[cut_frame].copy()
    cv2.putText(summary_frame, 'Scene Cut', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    scene_summary.append(summary_frame)

#### 6. **Result Visualization**

#### Step 6: Result Visualization
This step visualizes our results by displaying frames with detected scene cuts and selected segmented frames. We use OpenCV's `cv2.imshow()` function to show the frames sequentially. Each scene cut frame is displayed for 500 ms, and we similarly display a few segmented frames for illustration.

In [18]:
for frame in scene_summary:
    cv2.imshow("Scene Cut", frame)
    cv2.waitKey(500)  # Display each scene cut frame for 500 ms

for frame in segmented_frames[:10]:  # Display first 5 segmented frames
    cv2.imshow("Segmentation", frame)
    cv2.waitKey(500)
cv2.destroyAllWindows()